In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
list=([1,[1,1,1]],
      [2,[1,2,1]],
      [3,[1,1,1]],
      [4,[1,3,4]],
      [5,[1,1,1]],
      [6,[1,2,3]])

In [3]:
df= spark.createDataFrame(list, ['timestamp','vars'])
df.show()

+---------+---------+
timestamp| vars|
+---------+---------+
 1|[1, 1, 1]|
 2|[1, 2, 1]|
 3|[1, 1, 1]|
 4|[1, 3, 4]|
 5|[1, 1, 1]|
 6|[1, 2, 3]|
+---------+---------+

In [4]:

w=Window().orderBy("timestamp").rowsBetween(-1,-1)
w2=Window().orderBy("timestamp")
df.withColumn("vars2", F.lag("vars").over(w)).filter(F.col("timestamp") % 2 ==0)\
.withColumn("tmp", F.arrays_zip("vars", "vars2"))\
.withColumn("tmp", F.explode("tmp"))\
.withColumn("vars3", F.col("tmp.vars")+F.col("tmp.vars2"))\
.withColumn("vars4", F.col("vars3")+F.lag("vars3",3).over(w2))\
.show()

+---------+---------+---------+------+-----+-----+
timestamp| vars| vars2| tmp|vars3|vars4|
+---------+---------+---------+------+-----+-----+
 2|[1, 2, 1]|[1, 1, 1]|[1, 1]| 2| null|
 2|[1, 2, 1]|[1, 1, 1]|[2, 1]| 3| null|
 2|[1, 2, 1]|[1, 1, 1]|[1, 1]| 2| null|
 4|[1, 3, 4]|[1, 1, 1]|[1, 1]| 2| 4|
 4|[1, 3, 4]|[1, 1, 1]|[3, 1]| 4| 7|
 4|[1, 3, 4]|[1, 1, 1]|[4, 1]| 5| 7|
 6|[1, 2, 3]|[1, 1, 1]|[1, 1]| 2| 4|
 6|[1, 2, 3]|[1, 1, 1]|[2, 1]| 3| 7|
 6|[1, 2, 3]|[1, 1, 1]|[3, 1]| 4| 9|
+---------+---------+---------+------+-----+-----+

In [5]:
w = Window.orderBy(col('timestamp'))
result = df.withColumn('summed_vars',array(sum(col('vars')[0]).over(w),sum(col('vars')[1]).over(w),sum(col('vars')[2]).over(w))) \
           .withColumn('rnum',row_number().over(w))
result.filter(col('rnum')%2 == 0).select('timestamp','summed_vars').show()


+---------+-----------+
timestamp|summed_vars|
+---------+-----------+
 2| [2, 3, 2]|
 4| [4, 7, 7]|
 6|[6, 10, 11]|
+---------+-----------+

In [6]:
from pyspark.sql import functions as F


In [7]:
list=[[123,'01-01-2020','Y'],
      [234,'01-01-2020','Y'],
      [512,'01-01-2020','Y'],
      [111,'01-01-2020', 'Y']]
df=spark.createDataFrame(list,['sno_id','updt_dt','process_flag'])
df.show()

+------+----------+------------+
sno_id| updt_dt|process_flag|
+------+----------+------------+
 123|01-01-2020| Y|
 234|01-01-2020| Y|
 512|01-01-2020| Y|
 111|01-01-2020| Y|
+------+----------+------------+

In [8]:
array=df.agg(F.collect_list("sno_id").alias("list")).collect()[0][0]

In [9]:
array

Out[72]: [123, 234, 512, 111]

In [10]:
l=[]
for i in array:
  l.append(i[2])
  
l
  

Out[59]: [[112, 113], [123, 234, 512, 111]]

In [11]:
list=[[1,23,1],
[1,24,0],
[1,25,0],
[1,55,1],
[1,56,0],
[1,57,0],
[1,75,1],
[1,76,0],
[1,77,0],
[2,43,1],
[2,44,0],
[2,45,0]]

df=spark.createDataFrame(list,['id','timestamp','head_indices'])
df.show()

+---+---------+------------+
 id|timestamp|head_indices|
+---+---------+------------+
 1| 23| 1|
 1| 24| 0|
 1| 25| 0|
 1| 55| 1|
 1| 56| 0|
 1| 57| 0|
 1| 75| 1|
 1| 76| 0|
 1| 77| 0|
 2| 43| 1|
 2| 44| 0|
 2| 45| 0|
+---+---------+------------+

In [12]:
w=Window().partitionBy("id").orderBy("timestamp")
w2=Window().partitionBy("id").orderBy("head1")
w3=Window().partitionBy("id","head2").orderBy("head2")
df.withColumn("head1", F.when(F.col("head_indices")==1,F.lit(0)).otherwise(F.lit(1)))\
.withColumn("head2", F.dense_rank().over(w2))\
.withColumn("head3", F.row_number().over(w3))\
.orderBy("id","timestamp").show()

+---+---------+------------+-----+-----+-----+
 id|timestamp|head_indices|head1|head2|head3|
+---+---------+------------+-----+-----+-----+
 1| 23| 1| 0| 1| 1|
 1| 24| 0| 1| 2| 1|
 1| 25| 0| 1| 2| 2|
 1| 55| 1| 0| 1| 2|
 1| 56| 0| 1| 2| 3|
 1| 57| 0| 1| 2| 4|
 1| 75| 1| 0| 1| 3|
 1| 76| 0| 1| 2| 5|
 1| 77| 0| 1| 2| 6|
 2| 43| 1| 0| 1| 1|
 2| 44| 0| 1| 2| 1|
 2| 45| 0| 1| 2| 2|
+---+---------+------------+-----+-----+-----+

In [13]:
w=Window().partitionBy("id", "head_indices").orderBy("timestamp")
w2= Window().partitionBy("id","head3").orderBy("timestamp")
df.withColumn("head1", F.row_number().over(w))\
.withColumn("head3", F.when((F.col("head_indices")==0),F.when((F.col("head1")%2!=0),F.lit(0.1)).otherwise(F.lit(0))))\
.withColumn("seq_id", F.row_number().over(w2))\
.drop("head1","head3")\
.orderBy("id","timestamp").show()

+---+---------+------------+------+
 id|timestamp|head_indices|seq_id|
+---+---------+------------+------+
 1| 23| 1| 1|
 1| 24| 0| 1|
 1| 25| 0| 1|
 1| 55| 1| 2|
 1| 56| 0| 2|
 1| 57| 0| 2|
 1| 75| 1| 3|
 1| 76| 0| 3|
 1| 77| 0| 3|
 2| 43| 1| 1|
 2| 44| 0| 1|
 2| 45| 0| 1|
 2| 47| 0| 2|
+---+---------+------------+------+

In [14]:
w=Window().partitionBy("id", "head_indices").orderBy(F.col("timestamp"))
w2=Window().partitionBy("id").orderBy(F.col("timestamp")).rangeBetween(-5,0)
w3= Window().partitionBy("id").orderBy(F.col("timestamp").desc())
df.withColumn("head1", F.row_number().over(w))\
.withColumn("head2", F.when(F.col("head_indices")==1, F.col("head1")).otherwise(F.col("head_indices")))\
.withColumn("seq_id", F.first("head2").over(w2))\
.withColumn("head3", F.lead("timestamp").over(w3)-F.col("timestamp"))\
.orderBy("id","timestamp").show()

+---+---------+------------+-----+-----+------+-----+
 id|timestamp|head_indices|head1|head2|seq_id|head3|
+---+---------+------------+-----+-----+------+-----+
 1| 23| 1| 1| 1| 1| null|
 1| 24| 0| 1| 0| 1| -1|
 1| 25| 0| 2| 0| 1| -1|
 1| 55| 1| 2| 2| 2| -30|
 1| 56| 0| 3| 0| 2| -1|
 1| 57| 0| 4| 0| 2| -1|
 1| 75| 1| 3| 3| 3| -18|
 1| 76| 0| 5| 0| 3| -1|
 1| 77| 0| 6| 0| 3| -1|
 2| 43| 1| 1| 1| 1| null|
 2| 44| 0| 1| 0| 1| -1|
 2| 45| 0| 2| 0| 1| -1|
+---+---------+------------+-----+-----+------+-----+

In [15]:
w=Window().partitionBy("id", "head_indices").orderBy(F.col("timestamp"))
w2=Window().partitionBy("id").orderBy(F.col("timestamp")).rangeBetween(-5,0)
w3= Window().partitionBy("id").orderBy(F.col("timestamp").desc())
df.withColumn("head1", F.row_number().over(w))\
.withColumn("head2", F.when(F.col("head_indices")==1, F.col("head1")).otherwise(F.col("head_indices")))\
.orderBy("id","timestamp")\
.show()

+---+---------+------------+-----+-----+
 id|timestamp|head_indices|head1|head2|
+---+---------+------------+-----+-----+
 1| 23| 1| 1| 1|
 1| 24| 0| 1| 0|
 1| 25| 0| 2| 0|
 1| 55| 1| 2| 2|
 1| 56| 0| 3| 0|
 1| 57| 0| 4| 0|
 1| 75| 1| 3| 3|
 1| 76| 0| 5| 0|
 1| 77| 0| 6| 0|
 2| 43| 1| 1| 1|
 2| 44| 0| 1| 0|
 2| 45| 0| 2| 0|
+---+---------+------------+-----+-----+

In [16]:
-----------------------------------------------
| id |   timestamp    | head_indices | seq_id |
-----------------------------------------------
| 1  |     23         | 1            | 1      |
| 1  |     24         | 0            | 1      |
| 1  |     25         | 0            | 1      |
| 1  |     55         | 1            | 2      |
| 1  |     56         | 0            | 2      |
| 1  |     57         | 0            | 2      |
| 1  |     75         | 1            | 3      |
| 1  |     76         | 0            | 3      |
| 1  |     77         | 0            | 3      |
| 2  |     43         | 1            | 1      |
| 2  |     44         | 0            | 1      |
| 2  |     45         | 0            | 1      |
-----------------------------------------------

File "<command-2929972125296990>" , line 1 
 ----------------------------------------------- 
 ^
 SyntaxError : invalid syntax

In [17]:
df.selectExpr('*', 'sum(head_indices) OVER (Partition By id Order by timestamp) as seq_id').show()

+---+---------+------------+------+
 id|timestamp|head_indices|seq_id|
+---+---------+------------+------+
 1| 23| 1| 1|
 1| 24| 0| 1|
 1| 25| 0| 1|
 1| 55| 1| 2|
 1| 56| 0| 2|
 1| 57| 0| 2|
 1| 75| 1| 3|
 1| 76| 0| 3|
 1| 77| 0| 3|
 2| 43| 1| 1|
 2| 44| 0| 1|
 2| 45| 0| 1|
+---+---------+------------+------+